# **Задание 1**

Загружаем необходимые библиотеки

In [3]:
from pyspark.sql import SparkSession
import os
import shutil

Создаем SparkSession

In [4]:
spark = SparkSession.builder \
    .appName("JSON to DataFrame") \
    .getOrCreate()

Считываем файл 1.json и создаем Spark DataFrame

In [5]:
df = spark.read.json("/content/1.json")

Выводим первые 12 строк

In [6]:
df.show(12)

+-----------------+--------------------+---------------+--------------------+-----+----------+---------------+--------+------+--------------------+
|         category|         description|expiration_date|                name|price|product_id|production_date|quantity|rating|              status|
+-----------------+--------------------+---------------+--------------------+-----+----------+---------------+--------+------+--------------------+
|           Фрукты|Прошел сертификац...|     2025-03-09|              Яблоко| 2241|         1|     2013-08-18|      56|  1.41|           Delivered|
|             Мясо|Отличается высоки...|     2002-01-15|      Куриные грудки| 9467|         2|     1998-09-04|     472|  0.21|                 New|
|Молочные продукты|Способствует сниж...|     2020-05-01|               Шубат| 3812|         3|     2015-05-14|      11|  1.89|Available for Pickup|
|Молочные продукты|Прошел строгий ко...|     2023-03-03|            Пармезан| 9478|         4|     2018-03-02|  

Выводим количество колонок

In [7]:
column_count = len(df.columns)
print(f"Количество колонок: {column_count}")

Количество колонок: 10


Выводим статистические характеристики для всех колонок

In [8]:
df.describe().show()

+-------+--------+--------------------+---------------+----+-----------------+------------------+---------------+------------------+------------------+--------------------+
|summary|category|         description|expiration_date|name|            price|        product_id|production_date|          quantity|            rating|              status|
+-------+--------+--------------------+---------------+----+-----------------+------------------+---------------+------------------+------------------+--------------------+
|  count|     791|                 791|            791| 791|              791|               791|            791|               791|               791|                 791|
|   mean|    NULL|                NULL|           NULL|NULL| 5162.02402022756|             396.0|           NULL| 507.6321112515803|2.5377117572692813|                NULL|
| stddev|    NULL|                NULL|           NULL|NULL|2955.231614762311|228.48632344190756|           NULL|286.43209800060936|1.4

Удаляем указанную колонку: description

In [9]:
df = df.drop("description")

Создаем новую колонку: цена в $ (price * (94.24

In [10]:
df = df.withColumn("price_in_$", df["price"] * 94.24)

Записываем полученный набор данных в временную папку и проверяем, что файлы были записаны

In [11]:
output_dir = "/content/r_1_temp"
df.coalesce(1).write.csv(output_dir, header=True, mode='overwrite')
if os.path.exists(output_dir):
    print(f"Содержимое временной папки {output_dir}:")
    print(os.listdir(output_dir))
else:
    print(f"Папка {output_dir} не существует.")

Содержимое временной папки /content/r_1_temp:
['._SUCCESS.crc', '_SUCCESS', 'part-00000-ff8514e1-ee7e-495e-b0dd-75f3c259c62a-c000.csv', '.part-00000-ff8514e1-ee7e-495e-b0dd-75f3c259c62a-c000.csv.crc']


In [12]:
dst_file = "/content/r_1.csv" # Путь к целевому файлу

Проверяем, существует ли файл или папка с таким именем и удаляем, если это так

In [13]:
if os.path.exists(dst_file):
    if os.path.isdir(dst_file):
        shutil.rmtree(dst_file)
    else:
        os.remove(dst_file)

Объединяем все части в один файл

In [14]:
with open(dst_file, 'wb') as outfile:
    for file in os.listdir(output_dir):
        if file.startswith("part-") and file.endswith(".csv"):
            part_file_path = os.path.join(output_dir, file)
            with open(part_file_path, 'rb') as infile:
                shutil.copyfileobj(infile, outfile)

print(f"Файл успешно создан: {dst_file}")

Файл успешно создан: /content/r_1.csv


Удаляем временную папку

In [15]:
shutil.rmtree(output_dir)
print(f"Временная папка {output_dir} удалена.")

Временная папка /content/r_1_temp удалена.


In [16]:
spark.stop() # Завершаем сессию Spark

# **Задание 2**

Загружаем библиотеки

In [24]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, avg, min
import shutil
import os
import glob
import time

Создаем SparkSession

In [25]:
spark = SparkSession.builder \
    .appName("JSON to DataFrame") \
    .getOrCreate()

Считываем файл 1.json и создаем Spark DataFrame

In [26]:
df = spark.read.json("/content/2.json")

Выводим последние 5 строк

In [27]:
df.tail(5)

[Row(category='Electronics', description='Item 904', item_id=904, length=4888.74, price=1734.71, weight=6099.33, width=7732.28),
 Row(category='Musical Instruments', description='Item 905', item_id=905, length=270.35, price=2648.39, weight=2266.65, width=9327.94),
 Row(category='Art Supplies', description='Item 906', item_id=906, length=334.56, price=7888.39, weight=1222.35, width=89.4),
 Row(category='Musical Instruments', description='Item 907', item_id=907, length=2013.71, price=8507.64, weight=984.87, width=574.17),
 Row(category='Art Supplies', description='Item 908', item_id=908, length=487.62, price=74.2, weight=9007.79, width=1619.84)]

Отфильтруем данные, где length < 4622.0

In [28]:
filtered_df = df.filter(df["length"] < 4622.0)

Подсчитаем, сколько строк было отфильтровано на предыдущем шаге

In [29]:
filtered_count = filtered_df.count()
print(f"Количество отфильтрованных строк: {filtered_count}")

Количество отфильтрованных строк: 730


Выполняем группировку по полю category, для каждой категории рассчитав: количество элементов в группе, avg price, min width, min length

In [30]:
grouped_df = filtered_df.groupBy("category").agg(
    count("*").alias("count"),
    avg("price").alias("avg_price"),
    min("width").alias("min_width"),
    min("length").alias("min_length")
)

Записываем полученный набор данных в один файл

In [31]:
output_dir = "/content/temp_output"
final_output_file = "/content/r_2.json"
grouped_df.coalesce(1).write.json(output_dir, mode='overwrite')
time.sleep(1)

Проверяем, создана ли папка и есть ли в ней файлы

In [32]:
if os.path.exists(output_dir):
    part_file = os.path.join(output_dir, "part-*")
    part_file_paths = glob.glob(part_file)

    if part_file_paths:
        # Переименовываем файл в r_2.json
        shutil.move(part_file_paths[0], final_output_file)
        print(f"Данные успешно записаны в файл: {final_output_file}")
    else:
        print("Ошибка: не найден ни один файл в выходной директории.")
else:
    print("Ошибка: выходная директория не существует.")

Данные успешно записаны в файл: /content/r_2.json


Удаляем временную папку, если она существует

In [33]:
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

In [34]:
spark.stop() # Завершаем сессию Spark

# **Задание 3**

In [37]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean, count, when
import pandas as pd

In [38]:
spark = SparkSession.builder \
    .appName("CSV to DataFrame") \
    .getOrCreate()

Считываем файл 3.csv и создаем Spark DataFrame.

In [39]:
df = spark.read.csv("/content/3.csv", header=True, inferSchema=True)

Считаем количество пропусков

In [40]:
missing_counts = df.select([(count(col(c).isNull()).alias(c)) for c in df.columns])
missing_counts.show()

+---+----+-----+---+----+---+---+----+-----+-----+------+----+------+------+---+----+------+---+----+------+------+---+---+----+---+----+---+---+----+---+----+---+---+----+---+----+----+---+---+---+---+---+---+---+---+---+----+----+---+------+----+----+---+----+
|785|-690|null2|923|-344|638|651|-710|-5168|null9|-56810|-723|null12|null13|762|-405|-51616|949|-632|null19|null20|310|784|-927|575|-403|341|-22|-272|239|-365|401|529|-108|694|-262|-388|886|394| 69|563|252| 90| -2|287|-57|-270|-876|156|-56849|-244|-569|737|-867|
+---+----+-----+---+----+---+---+----+-----+-----+------+----+------+------+---+----+------+---+----+------+------+---+---+----+---+----+---+---+----+---+----+---+---+----+---+----+----+---+---+---+---+---+---+---+---+---+----+----+---+------+----+----+---+----+
|289| 289|  289|289| 289|289|289| 289|  289|  289|   289| 289|   289|   289|289| 289|   289|289| 289|   289|   289|289|289| 289|289| 289|289|289| 289|289| 289|289|289| 289|289| 289| 289|289|289|289|289|289|289|2

Обработываем пропуски: заполняем средним по колонке

In [41]:
for column in df.columns:
    mean_value = df.select(mean(col(column))).first()[0]
    df = df.fillna({column: mean_value})

Создаем новый столбец с количеством положительных значений

In [42]:
positive_count_column = sum(when(col(c) > 0, 1).otherwise(0) for c in df.columns)
df = df.withColumn("positive_count", positive_count_column)

Производим фильтрацию данных: оставляем строки, в которых количество положительных элементов больше 0.

In [43]:
df_filtered = df.where(col("positive_count") > 0)

Подсчитываем статистику по каждой колонке и сохраняем в отдельный df

In [44]:
stats_df = df_filtered.describe()

Преобразуем DataFrame в Pandas для записи в файл

In [45]:
pandas_df = df_filtered.toPandas()

Записываем полученный DataFrame в файл r_3.csv

In [46]:
pandas_df.to_csv("/content/r_3.csv", index=False)

In [47]:
spark.stop() # Завершаем сессию Spark